El paquete de _Python_ [scikit-learn](http://scikit-learn.org) (_sklearn_ en lo que sigue) proporciona un marco de trabajo para el aprendizaje automático.

# Aprendizaje supervisado

Para ilustrar el concepto de aprendizaje supervisado vamos a usar el conjunto de datos [_Car Evaluation_](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation) del repositorio [UCI](http://archive.ics.uci.edu/ml/). Este conjunto de datos contiene información acerca de la idoneidad de una serie de coches, en función de los siguientes atributos:
* _buying_: precio de compra. Posibles valores: vhigh, high, med, low.
* _maint_: coste de mantenimiento. Posibles valores: vhigh, high, med, low.
* _doors_: número de puertas. Posibles valores: 2, 3, 4, 5more.
* _persons_: número de asientos. Posibles valores: 2, 4, more.
* *lug\_boot*: tamaño del maletero. Posibles valores: small, med, big.
* _safety_: nivel de seguridad estimada. Posibles valores: low, med, high.

La idoneidad de cada coche se indica mediante el atributo _acceptability_, que los clasifica como _unacc_, _acc_, _good_ o _vgood_.

Para leer los datos desde el fichero `cars.csv` que se proporciona se pueden evaluar las siguientes expresiones ([_Pandas_](http://pandas.pydata.org/) y [_NumPy_](http://www.numpy.org/) son paquetes de _Python_ para análisis de datos y cálculo científico, respectivamente):

In [ ]:
import pandas
import numpy

cars = pandas.read_csv('cars.csv', header=None,
                       names=['buying', 'maint', 'doors', 'persons',
                              'lug_boot', 'safety', 'acceptability'])
print(cars.shape)  # Número de filas y columnas
cars.head(10)

_sklearn_ no puede trabajar directamente con el conjunto de datos anterior, ya que asume que los valores de las variables discretas están codificadas con números enteros. Para transformar los datos a un formato adecuado ofrece diversas operaciones de preprocesamiento, entre las que se encuentra _LabelEncoder_.

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()  # Creamos un codificador de etiquetas
le.fit(cars['buying'])  # Calculamos la codificación de cada valor
print(le.classes_)
print(le.transform(['vhigh', 'med', 'high', 'low', 'vhigh']))  # Codificamos los valores
print(le.inverse_transform([3, 2, 0, 1, 3]))  # Descodificamos los códigos

Hay que repetir el esquema anterior para cada columna de la tabla de datos. Además, conservaremos los codificadores de cada columna para poder usar la misma codificación con nuevos ejemplos.

In [ ]:
codificadores = []
cars_codificado = pandas.DataFrame()
for variable, valores in cars.iteritems():
    le = preprocessing.LabelEncoder()
    le.fit(valores)
    print('Codificación de valores para {}: {}'.format(variable, le.classes_))
    codificadores.append(le)
    cars_codificado[variable] = le.transform(valores)

cars_codificado.head(10)

# Si no es necesario conservar los codificadores, la siguiente es una manera más
# directa de codificar las variables
# le = preprocessing.LabelEncoder()
# cars_codificado = cars.apply(le.fit_transform, axis=0)

Una vez codificadas las variables, es necesario separar el conjunto de datos en dos: un conjunto de entrenamiento, que se usará para generar los distintos modelos; y un conjunto de prueba, que se usará para comparar los distintos modelos.

Un detalle a tener en cuenta es que la distribución de ejemplos en las distintas clases de aceptabilidad no es uniforme: hay 1210 coches (un 70.023 % del total) clasificados como inaceptables (`unacc`), 384 coches (22.222 %) clasificados como aceptables (`acc`), 69 coches ( 3.993 %) clasificados como buenos (`good`) y 65 coches ( 3.762 %) clasificados como muy buenos (`vgood`).

Es conveniente, por tanto, que la separación de los ejemplos se realice de manera estratificada, es decir, intentando mantener la proporción anterior tanto en el conjunto de entrenamiento como en el de prueba.

Para dividir un conjunto de datos en un subconjunto de entrenamiento y otro de prueba, _sklearn_ proporciona la función `train_test_split`.

In [ ]:
from sklearn import model_selection

print('Codificación:', codificadores[-1].classes_)
print(cars_codificado.shape[0])  # Cantidad total de ejemplos
print(cars_codificado['acceptability'].value_counts(
        normalize=True, sort=False))  # Frecuencia total de cada clase de aceptabilidad

cars_entrenamiento, cars_prueba = model_selection.train_test_split(
    cars_codificado, test_size=.33, random_state=12345,
    stratify=cars_codificado['acceptability'])

# Comprobamos que el conjunto de prueba contiene el 33 % de los datos, en la misma proporción
# con respecto a la variable objetivo
print(cars_prueba.shape[0], 1728 * .33)
print(cars_prueba['acceptability'].value_counts(
        normalize=True, sort=False))

# Comprobamos que el conjunto de entrenamiento contiene el resto de los datos, en la misma
# proporción con respecto a la variable objetivo
print(cars_entrenamiento.shape[0], 1728 * (1 - .33))
print(cars_entrenamiento['acceptability'].value_counts(
        normalize=True, sort=False))

Para realizar aprendizaje supervisado en _sklearn_ basta crear una instancia de la clase de objetos que implemente el modelo que se quiera utilizar (árboles de decisión, _naive_ Bayes, _kNN_, etc.).

Cada una de estas instancias dispondrá de los siguientes métodos:
* El método `fit` permite entrenar el modelo, dados __por separado__ el conjunto de ejemplos de entrenamiento y la clase de cada uno de estos ejemplos.
* El método `predict` permite clasificar un nuevo ejemplo una vez entrenado el modelo.
* El método `score` calcula el rendimiento del modelo, dados __por separado__ el conjunto de ejemplos de prueba y la clase de cada uno de estos ejemplos.

Un requisito para poder continuar es separar los conjuntos de datos `cars_entrenamiento` y `cars_prueba` en los valores de los atributos por un lado y la clasificación por otro.

In [ ]:
datos_entrenamiento = cars_entrenamiento.loc[:, 'buying':'safety']
clases_entrenamiento = cars_entrenamiento['acceptability']

datos_prueba = cars_prueba.loc[:, 'buying':'safety']
clases_prueba = cars_prueba['acceptability']

### Árboles de decisión

_sklearn_ implementa los árboles de decisión clasificadores como instancias de la clase `DecisionTreeClassifier`.

Desafortunadamente, son árboles de decisión binarios construidos asumiendo atributos continuos y mediante un algoritmo distinto a _ID3_, que no está implementado.

En http://scikit-learn.org/stable/modules/tree.html se puede encontrar información acerca de los árboles de decisión implementados en _sklearn_.

### _Naive_ Bayes

_sklearn_ implementa _naive_ Bayes para atributos discretos mediante instancias de la clase `MultinomialNB`.

In [ ]:
from sklearn import naive_bayes

clasif_NB = naive_bayes.MultinomialNB(alpha=1.0)  # alpha es el tamaño muestral equivalente

La utilización de la instancia construida requiere que los atributos que caracterizan los ejemplos sean binarios. Podemos transformar nuestros atributos multinomiales en atributos binarios mediante el preprocesador `OneHotEncoder` de _sklearn_.

Este preprocesador permite convertir una variable con $n$ posibles valores en $n$ variables binarias mutuamente excluyentes que codifican el valor que para esa variable tiene el ejemplo.

In [ ]:
ohe = preprocessing.OneHotEncoder(sparse = False)
# En las expresiones siguientes, el método values proporciona los datos como un vector de
# numpy y el método reshape lo transforma entonces a una matriz con una columna y tantas
# filas como sea necesario. Esto debe hacerse ya que el preprocesador OneHotEncoder solo
# trabaja con matrices.
print(cars_codificado['buying'].values)
print(cars_codificado['buying'].values.reshape(-1, 1))
ohe.fit(cars_codificado['buying'].values.reshape(-1,1))
ohe.transform(cars_codificado['buying'].values.reshape(-1, 1))

In [ ]:
ohe = preprocessing.OneHotEncoder(sparse = False)
# El método fit_transform realiza un ajuste a partir de una matriz de datos seguido de una
# transformación de esos mismos datos.
datos_entrenamiento_nb = ohe.fit_transform(datos_entrenamiento)
datos_prueba_nb = ohe.fit_transform(datos_prueba)

Ya estamos en condiciones de poder entrenar el modelo _naive_ Bayes.

In [ ]:
clasif_NB.fit(datos_entrenamiento_nb, clases_entrenamiento)

Las siguientes expresiones muestran las cuentas realizadas y (los logaritmos de) las probabilidades aprendidas por el modelo.

In [ ]:
print(clasif_NB.class_count_)
print(clasif_NB.class_log_prior_)
print(clasif_NB.feature_count_)
print(clasif_NB.feature_log_prob_)

El método `predict` devuelve la clase predicha por el modelo para un nuevo ejemplo y el método `score` el rendimiento sobre un conjunto de datos de prueba.

In [ ]:
nuevo_ejemplo = ['vhigh', 'vhigh', '3', 'more', 'big', 'high']
# Codificamos los valores de los atributos
nuevo_ejemplo_codif = [le.transform([valor])
                       for valor, le in zip(nuevo_ejemplo, codificadores)]
nuevo_ejemplo_codif = numpy.reshape(nuevo_ejemplo_codif, (1, -1))
print(nuevo_ejemplo_codif)
# Transformamos los atributos a codificación binaria
nuevo_ejemplo_nb = ohe.transform(nuevo_ejemplo_codif)
print(nuevo_ejemplo_nb)
# Predecimos la clase
clase_nuevo_ejemplo = clasif_NB.predict(nuevo_ejemplo_nb)
print(codificadores[-1].inverse_transform(clase_nuevo_ejemplo))

In [ ]:
# Calculamos la fracción de clases correctamente predichas para el conjunto de datos de prueba
clasif_NB.score(datos_prueba_nb, clases_prueba)

### kNN

_sklearn_ implementa _kNN_ como instancias de la clase `KNeighborsClassifier`. En http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html se puede encontrar una descripción de las distancias actualmente implementadas que se podrían usar.

In [ ]:
from sklearn import neighbors

clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=5, metric='hamming')

Entrenamos el modelo.

In [ ]:
clasif_kNN.fit(datos_entrenamiento, clases_entrenamiento)

El método `kneighbors` permite encontrar los (índices de los) $k$ vecinos más cercanos de los ejemplos proporcionados, así como las distancias a las que se encuentran.

In [ ]:
distancias, vecinos = clasif_kNN.kneighbors(nuevo_ejemplo_codif)
print(nuevo_ejemplo_codif)
print(datos_entrenamiento.iloc[vecinos[0]])
print(distancias[0])

El método `predict` devuelve la clase predicha por el modelo para un nuevo ejemplo y el método `score` el rendimiento sobre un conjunto de datos de prueba.

In [ ]:
clase_nuevo_ejemplo = clasif_kNN.predict(nuevo_ejemplo_codif)
print(codificadores[-1].inverse_transform(clase_nuevo_ejemplo))

In [ ]:
clasif_kNN.score(datos_prueba, clases_prueba)

## Solicitudes de admisión en guarderías

El fichero de datos `nursery.csv` proporciona un conjunto de datos acerca de la evaluación de solicitudes de admisión en guarderías, en función de los siguientes atributos:
* _parents_, con posibles valores: usual, pretentious, great_pret.
* *has\_nurs*, con posibles valores: proper, less_proper, improper, critical, very_crit.
* _form_, con posibles valores: complete, completed, incomplete, foster.
* _children_, con posibles valores: 1, 2, 3, more.
* _housing_, con posibles valores: convenient, less_conv, critical.
* _finance_, con posibles valores: convenient, inconv.
* _social_, con posibles valores: non-prob, slightly_prob, problematic.
* _health_, con posibles valores: recommended, priority, not_recom.

Los datos provienen de un sistema experto de decisión usado durante varios años de la década de los 80 en Liubliana (Eslovenia), que se desarrolló para poder proporcionar una explicación objetiva a las solicitudes rechazadas.

La evaluación de cada solicitud se indica mediante el atributo _evaluation_, que los clasifica como *not\_recom*, _recommend_, *very\_recom*, _priority_ o *spec\_prior*.

El objetivo es aprender a partir de los datos un modelo que prediga de la mejor forma posible cómo se evaluará una solicitud de admisión a partir de los valores de los atributos anteriores. Para ello se pide seguir los siguientes pasos:
* Leer los datos a partir del fichero `nursery.csv`.

* Codificar los datos con números enteros.

* Dividir el conjunto de datos en tres subconjuntos: un subconjunto `nursery_cv` (20 % de los datos) para determinar los mejores parámetros de los modelos mediante validación cruzada; un subconjunto `nursery_entrenamiento` (60 % de los datos) para entrenar los modelos con esos parámetros; un subconjunto `nursery_prueba` (20 % de los datos) para comparar los modelos a partir de su rendimiento sobre él.

* Para cada uno de los subconjuntos anteriores, separar los datos correspondientes a los valores de los atributos de cada ejemplo de los datos correspondientes a la clasificación de cada ejemplo.

* Dividir el subconjunto `nursery_cv` en 10 subconjuntos y usarlos para calcular mediante validación cruzada el rendimiento promedio del algoritmo _naive_ Bayes. Considerar para el tamaño muestral equivalente cada uno de los valores enteros entre 1 y 10 y determinar cuál es el que proporciona mejor rendimiento promedio.

In [ ]:
from sklearn import model_selection
from sklearn import naive_bayes


* Con el mejor tamaño muestral equivalente determinado en el punto anterior, entrenar el algoritmo _naive_ Bayes con el subconjunto `nursery_entrenamiento` y calcular el rendimiento sobre el subconjunto `nursery_prueba`.

* Dividir el subconjunto `nursery_cv` en 10 subconjuntos y usarlos para calcular mediante validación cruzada el rendimiento promedio del algoritmo _kNN_. Considerar para $k$ cada uno de los valores enteros entre 1 y 10 y determinar cuál es el que proporciona mejor rendimiento promedio.

In [ ]:
from sklearn import neighbors


* Con el mejor $k$ determinado en el punto anterior, entrenar el algoritmo _kNN_ con el subconjunto `nursery_entrenamiento` y calcular el rendimiento sobre el subconjunto `nursery_prueba`.

* ¿Cuál de los dos modelos construidos en los puntos anteriores realiza mejores predicciones acerca de la evaluación de las solicitudes de admisión?

__Nota__: la función `cross_val_score` del módulo `model_selection` de _sklearn_ implementa el procedimiento de validación cruzada. Admite, entre otros, los siguientes argumentos:
* _estimator_: modelo a evaluar.
* _X_: array con los valores de los atributos de los ejemplos.
* _y_: array con las clasificaciones de los ejemplos.
* _cv_: número de subconjuntos en los que dividir los datos.

Devuelve un array con el rendimiento del modelo sobre cada subconjunto, una vez entrenado con los ejemplos del resto de subconjuntos.

Para más información acerca de la implementación en _sklearn_ del método de validación cruzada puede consultarse http://scikit-learn.org/stable/modules/cross_validation.html.

Respuesta: